In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Importaciones específicas para evitar el NameError
from sklearn.model_selection import train_test_split  # pyright: ignore[reportMissingImports]
from sklearn.preprocessing import StandardScaler  # pyright: ignore[reportMissingImports]
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoLars, BayesianRidge  # pyright: ignore[reportMissingImports]
from sklearn.metrics import mean_squared_error, r2_score  # pyright: ignore[reportMissingImports]

print("Librerías cargadas correctamente.")


Librerías cargadas correctamente.


In [4]:
# 1. Definir la ruta del archivo
archivo_path = "Chat_Vecinos.txt"

def crear_dataset_whatsapp(path):
    datos = []
    # Expresión regular para detectar el inicio de un mensaje: Fecha, Hora - Usuario:
    patron_inicio = r'^(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2})\s-\s([^:]+):\s'
    
    ultimo_mensaje = None

    with open(path, 'r', encoding='utf-8') as f:
        for linea in f:
            linea = linea.strip()
            if not linea: continue # Saltar líneas vacías
            
            match = re.match(patron_inicio, linea)
            
            if match:
                # Si hay un match, es un mensaje nuevo
                fecha, hora, usuario = match.groups()
                # El mensaje real empieza después de ": " (que es el fin del patrón)
                contenido_mensaje = linea[match.end():]
                
                ultimo_mensaje = {
                    'Fecha': fecha,
                    'Hora': hora,
                    'Usuario': usuario,
                    'Mensaje': contenido_mensaje
                }
                datos.append(ultimo_mensaje)
            else:
                # Si no hay match y ya tenemos un mensaje previo, es un salto de línea
                if ultimo_mensaje:
                    ultimo_mensaje['Mensaje'] += " " + linea
                # Si no hay mensaje previo (como avisos de sistema), lo ignoramos

    return pd.DataFrame(datos)

# 2. Ejecutar la función y crear el dataset
df = crear_dataset_whatsapp(archivo_path)

# 3. Limpieza y visualización
# Convertir la columna Fecha a formato datetime para poder hacer análisis temporal
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)

print(f"Dataset creado con {len(df)} registros.")
print(df.head(10))

# Guardar el resultado
# df.to_excel('Mantenimiento_E2.xlsx', index=False)

Dataset creado con 375 registros.
       Fecha   Hora                Usuario  \
0 2025-04-25  18:15       +52 55 1859 7534   
1 2025-04-25  18:48       +52 55 1859 7534   
2 2025-04-25  20:39  Sr. Felipe Vecino 501   
3 2025-04-26   8:36  Sr. Felipe Vecino 501   
4 2025-04-26   9:53       +52 55 5401 9386   
5 2025-04-26  10:54       +52 55 3476 5535   
6 2025-04-26  14:30       +52 55 1859 7534   
7 2025-04-28  13:48           Jessy Vecina   
8 2025-05-05  13:13       +52 55 2968 5219   
9 2025-05-05  13:13       +52 55 2968 5219   

                                             Mensaje  
0  Buenos tardes representantes ya llegaron las  ...  
1                               <Multimedia omitido>  
2  Vecinos estamos esperando para la junta de hoy...  
3  Se quedo de acuerdo en la junta del E2 qué se ...  
4                                         De acuerdo  
5                                Bien día de acuerdo  
6                          Buenos tardes, de acuerdo  
7                  

/var/folders/mg/n7s2vkx519n0rm0lmvcjfn_40000gn/T/ipykernel_31837/3252972637.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)


¿Que tengo? Un chat vecinal donde se presentan mensajes como avisos de juntas, quejas o dudas sobre la convivencia en el edificio
Propuesta: 
*Variable objetivo Discreta: Prioridad. Se puede crear un sistema de alertas. Para que el administrador pueda atender problemas criticos con la prioridad correspondiente.
0: Baja (saludos, agradecimientos, confirmaciones)
1: Media (Dudas sobre pagos, avisos de juntas, temas administrativos)
2: Alta (Reportes de falta de agua, fugas, vigilancia, conflicto o mantenimiento)

Podemos usar Red Elastica que combina lasso y ridge


*Variable objetiva Continua: Indice de participacion: 
Medir que tipo de anuncios son los que a los vecinos les ponen atención. Lo que permite que administrativamente tener respuestas o argumentos para los anuncios que causaran "conflicto" (usamos Lasso). 
Tambien se puede saber cuando enviar mensajes para obtener más o menos atención. (Ridge o Bayesiana)
Seleccion de temas criticos: Podemos saber que temas tienen la mayor atencion del foro y con eso priorizar las juntas (forward selection)





In [7]:
# --- PARTE A: Definición de la Variable Objetivo Discreta (Prioridad) ---
def asignar_prioridad(mensaje):
    msg = str(mensaje).lower()
    # Prioridad 2: Alta (Emergencias y Seguridad)
    if any(w in msg for w in ['agua', 'fuga', 'vigilancia', 'robo', 'emergencia', 'luz', 'urgente', 'seguridad']):
        return 2
    # Prioridad 1: Media (Administración y Trámites)
    elif any(w in msg for w in ['pago', 'junta', 'administracion', 'duda', 'recibo', 'mantenimiento', 'deposito']):
        return 1
    # Prioridad 0: Baja (Social)
    else:
        return 0

# --- PARTE B: Definición de la Variable Objetivo Continua (Índice de Participación) ---
# En un entorno real, esto se mide por el número de respuestas. 
# Aquí lo estimamos por la relevancia del tema y la complejidad del mensaje.
def calcular_participacion(row):
    score = len(str(row['Mensaje'])) * 0.05  # Base por longitud
    if row['Prioridad'] == 2: score *= 2.0   # Los temas urgentes generan más interacción
    if row['Prioridad'] == 1: score *= 1.3   # Los temas administrativos generan interacción media
    return round(score, 2)

# Aplicamos las funciones al DataFrame
df['Prioridad'] = df['Mensaje'].apply(asignar_prioridad)
df['Indice_Participacion'] = df.apply(calcular_participacion, axis=1)

# Variables adicionales para los modelos (Features)
df['Hora_Int'] = df['Hora'].apply(lambda x: int(x.split(':')[0]))
df['Longitud_Msg'] = df['Mensaje'].apply(lambda x: len(str(x)))

print("Variables objetivo construidas.")
print(df[['Mensaje', 'Prioridad', 'Indice_Participacion']].head(10))

Variables objetivo construidas.
                                             Mensaje  Prioridad  \
0  Buenos tardes representantes ya llegaron las  ...          0   
1                               <Multimedia omitido>          0   
2  Vecinos estamos esperando para la junta de hoy...          1   
3  Se quedo de acuerdo en la junta del E2 qué se ...          2   
4                                         De acuerdo          0   
5                                Bien día de acuerdo          0   
6                          Buenos tardes, de acuerdo          0   
7                  Buenas tardes , enterada. Gracias          0   
8  Buenas tardes soy del  depto 404,  quiero preg...          0   
9                               <Multimedia omitido>          0   

   Indice_Participacion  
0                  3.90  
1                  1.00  
2                  3.51  
3                 22.00  
4                  0.50  
5                  0.95  
6                  1.25  
7                  1.6

In [10]:
# Variables que el modelo usará para predecir la participación
X = df[['Hora_Int', 'Longitud_Msg', 'Prioridad']] 
y = df['Indice_Participacion']

# División Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalado (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Set de entrenamiento: {X_train.shape[0]} muestras")

Set de entrenamiento: 262 muestras
